###Demonstrating cleaning data in Python & creating new SQL relational database:

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import sqlalchemy

In [3]:
engine = sqlalchemy.create_engine('sqlite:///res_data.db')

In [4]:
%load_ext sql

In [5]:
%sql sqlite:///res_data.db

In [65]:
data = pd.read_excel("2020-residential.xlsx")
data.head()

,Neighborhood\nCode,Account #,PARCELNB,PROPERTY_ADDRESS,LOCCITY,SUBNAME,MULTIPLE_BLDGS,ACCOUNT_TYPE,BLDG1_DESCRIPTION,BLDG1_DESIGN,...,OWNER_NAME,CARE_OF,MAILING_ADDR1,MAILING_ADDR2,CITY,STATE,ZIPCODE,LAND_VALUE,BLDG_VALUE,EXTRA_FEATURE_VALUE
0,103,R0071420,146136416013,1042 8TH ST,BOULDER,ROSE HILL - BO,YES,RESIDENTIAL,SINGLE FAM RES IMPROVEMENTS,2-3 Story,...,2B9 8TH STREET LLC,NaN,1623 CENTRAL AVE STE 204,NaN,CHEYENNE,WY,82001,1035000,807000,0
1,107,R0000908,146125304001,2385 4TH ST,BOULDER,MAPLETON PARK - BO,NO,RESIDENTIAL,SINGLE FAM RES IMPROVEMENTS,2-3 Story,...,DUFOUR JEFFREY W & CECILIA C DALEY,NaN,2385 4TH ST,NaN,BOULDER,CO,80302,1102000,915600,0
2,155,R0110294,157530104008,1486 SEIBERT CT,SUPERIOR,ROCK CREEK RANCH FLG 4A - SU,NO,RESIDENTIAL,SINGLE FAM RES IMPROVEMENTS,2-3 Story,...,TROTTER ERICA & JOHN RYZIW,NaN,1486 S SEIBERT CT,NaN,SUPERIOR,CO,80027-8024,358000,256700,0
3,158,R0069754,157707417048,1400 ROCKMONT CIR,BOULDER,DEVILS THUMB 4 - BO,NO,RESIDENTIAL,SINGLE FAM RES IMPROVEMENTS,Split-level,...,TSK VENTURES LLC,NaN,10705 ALEXANDER MILL DR,NaN,CHARLOTTE,NC,28277,690000,266000,0
4,440,R0126278,146526314006,1609 HOLEMAN DR,ERIE,ARAPAHOE RIDGE REPLAT A - ER,NO,RESIDENTIAL,SINGLE FAM RES IMPROVEMENTS,2-3 Story,...,HAYES ADAM R,NaN,1609 HOLEMAN DR,NaN,ERIE,CO,80516,76000,385300,0


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850 entries, 0 to 2849
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Neighborhood
Code     2850 non-null   int64         
 1   Account #             2850 non-null   object        
 2   PARCELNB              2850 non-null   object        
 3   PROPERTY_ADDRESS      2850 non-null   object        
 4   LOCCITY               2850 non-null   object        
 5   SUBNAME               2850 non-null   object        
 6   MULTIPLE_BLDGS        2850 non-null   object        
 7   ACCOUNT_TYPE          2850 non-null   object        
 8   BLDG1_DESCRIPTION     2850 non-null   object        
 9   BLDG1_DESIGN          2850 non-null   object        
 10  BLDG1_YEAR_BUILT      2850 non-null   int64         
 11  BEDROOMS              2850 non-null   int64         
 12  FULL_BATHS            2850 non-null   int64         
 13  THREE_QTR_BATHS   

In [67]:
#drop columns concerning purchaser info - dataset focused on specifics of properties of building sold
data = data.drop(['PARCELNB', 'SUBNAME', 'RECEPTION_NO', 'Grantor 1', 'Grantee 1', 'OWNER_NAME', 'CARE_OF',
                     'MAILING_ADDR1', 'MAILING_ADDR2', 'CITY', 'STATE', 'ZIPCODE'], axis='columns')

In [68]:
#check for any null values
data.isna().sum()

Neighborhood\nCode      0
Account #               0
PROPERTY_ADDRESS        0
LOCCITY                 0
MULTIPLE_BLDGS          0
ACCOUNT_TYPE            0
BLDG1_DESCRIPTION       0
BLDG1_DESIGN            0
BLDG1_YEAR_BUILT        0
BEDROOMS                0
FULL_BATHS              0
THREE_QTR_BATHS         0
HALF_BATHS              0
ABOVE_GROUND_SQFT       0
FINISHED_BSMT_SQFT      0
UNFINISHED_BSMT_SQFT    0
GARAGE_SQFT             0
FINISHED_GARAGE_SQFT    0
STUDIO_SQFT             0
OTHER_BLDGS             0
SALE_DATE               0
SALE_PRICE              0
LAND_VALUE              0
BLDG_VALUE              0
EXTRA_FEATURE_VALUE     0
dtype: int64

In [69]:
#rename columns for easier writing & understanding
data.rename(columns = {"Neighborhood\nCode": "NCODE", "Account #": "ACCT#", "BLDG1_DESCRIPTION": "DESCRIP", 
                           "BLDG1_DESIGN": "DESIGN", "BLDG1_YEAR_BUILT": "YEAR_BUILT"}, inplace=True)

In [70]:
#change all string values to upper-case for conformity
data = data.apply(lambda x: x.astype(str).str.upper())

In [71]:
#Account numbers should be unique - if not likely duplicate data due to LLC change (not new sale) or input mistake
#Check length of full database
print("Full Data: " + str(len(data)))
#Check length of unique ACCT#
acct = data["ACCT#"]
acct_set = set(acct)
print("Unique Account: " + str(len(acct_set)))

Full Data: 2850
Unique Account: 2798


In [73]:
#Shows 52 missing rows, check for duplicates
dup_rows = data[data["ACCT#"].duplicated()]
print("Duplicate Rows: " + str(len(dup_rows)))
#shows 52 rows are duplicates

Duplicate Rows: 52


In [74]:
#Drop duplicate rows
data = data.drop_duplicates(subset=["ACCT#"], keep = 'first')
data.shape

(2798, 25)

In [75]:
#Transform categorical variables into numeric representation - starting with LOCCITY
le = preprocessing.LabelEncoder()
data["LOCCITY"] = le.fit_transform(data["LOCCITY"])

In [76]:
#Save list of categorical variables
city_list = list(le.classes_)
city_list

['BOULDER',
 'ERIE',
 'LAFAYETTE',
 'LONGMONT',
 'LOUISVILLE',
 'LYONS',
 'NEDERLAND',
 'SUPERIOR',
 'UNINCORPORATED',
 'WARD']

In [77]:
#Convert list to dataframe
city_df = pd.DataFrame(city_list)
city_df

,0
0,BOULDER
1,ERIE
2,LAFAYETTE
3,LONGMONT
4,LOUISVILLE
5,LYONS
6,NEDERLAND
7,SUPERIOR
8,UNINCORPORATED
9,WARD


In [78]:
#Save dataframe as new SQL Table in database
city_df.to_sql('city', engine, index=True)

In [79]:
#Check to make sure table imported correctly
%sql SELECT * FROM city

 * sqlite:///res_data.db
Done.


index,0
0,BOULDER
1,ERIE
2,LAFAYETTE
3,LONGMONT
4,LOUISVILLE
5,LYONS
6,NEDERLAND
7,SUPERIOR
8,UNINCORPORATED
9,WARD


In [80]:
#Rename table columns
%sql ALTER TABLE city RENAME COLUMN 'Index' TO 'city_code';
%sql ALTER TABLE city RENAME COLUMN '0' TO 'city_name';

 * sqlite:///res_data.db
Done.
 * sqlite:///res_data.db
Done.


1


In [81]:
#Check table again to make sure correct
%sql SELECT * FROM city

 * sqlite:///res_data.db
Done.


city_code,city_name
0,BOULDER
1,ERIE
2,LAFAYETTE
3,LONGMONT
4,LOUISVILLE
5,LYONS
6,NEDERLAND
7,SUPERIOR
8,UNINCORPORATED
9,WARD


In [82]:
#Continue converting categorical variables - multiple_bldgs
data["MULTIPLE_BLDGS"] = le.fit_transform(data["MULTIPLE_BLDGS"])
bldg_list = list(le.classes_)
bldg_df = pd.DataFrame(bldg_list)
bldg_df.to_sql('multiple_bldgs', engine, index=True)

In [83]:
%sql SELECT * FROM multiple_bldgs

 * sqlite:///res_data.db
Done.


index,0
0,NO
1,YES


In [84]:
%%sql 
ALTER TABLE multiple_bldgs RENAME COLUMN 'Index' TO 'mb_code';
ALTER TABLE multiple_bldgs RENAME COLUMN '0' TO 'Y/N';

 * sqlite:///res_data.db
Done.
Done.


1


In [85]:
%sql SELECT * FROM multiple_bldgs

 * sqlite:///res_data.db
Done.


mb_code,Y/N
0,NO
1,YES


In [87]:
#Convert account_type
data["ACCOUNT_TYPE"] = le.fit_transform(data["ACCOUNT_TYPE"])
at_list = list(le.classes_)
at_df = pd.DataFrame(at_list)
at_df.to_sql('account_type', engine, index=True)

In [88]:
%%sql
ALTER TABLE account_type RENAME COLUMN 'Index' TO 'acct_type_code';
ALTER TABLE account_type RENAME COLUMN '0' TO 'acct_type_desc';

 * sqlite:///res_data.db
Done.
Done.


1


In [89]:
%sql SELECT * FROM account_type

 * sqlite:///res_data.db
Done.


acct_type_code,acct_type_desc
0,AFFORDABLE RES
1,AGRICULTURAL
2,EXEMPT
3,MANUFACTURED HOME
4,PART EXEMPT
5,RESIDENT LAND
6,RESIDENTIAL
7,RESIDENTIAL CONDO


In [90]:
#Convert description
data["DESCRIP"] = le.fit_transform(data["DESCRIP"])
desc_list = list(le.classes_)
desc_df = pd.DataFrame(desc_list)
desc_df.to_sql('description', engine, index=True)

In [91]:
%%sql
ALTER TABLE description RENAME COLUMN 'Index' TO 'desc_code';
ALTER TABLE description RENAME COLUMN '0' TO 'desc_full'

 * sqlite:///res_data.db
Done.
Done.


1


In [92]:
%sql SELECT * FROM description

 * sqlite:///res_data.db
Done.


desc_code,desc_full
0,CONDOS-IMPROVEMENTS
1,DUP/TRIPLEX IMPROVEMENTS
2,EX CHARITABLE RES IMPS
3,FARM/RANCH RESIDENTIAL IMPROVEMENTS
4,MANUFACTURED HOUSING IMPROVEMENTS
5,SINGLE FAM RES IMPROVEMENTS


In [ ]:
#Convert design

In [93]:
data["DESIGN"] = le.fit_transform(data["DESIGN"])
design_list = list(le.classes_)
design_df = pd.DataFrame(design_list)
design_df.to_sql('design', engine, index=True)

In [94]:
%%sql
ALTER TABLE design RENAME COLUMN 'Index' TO 'design_code';
ALTER TABLE design RENAME COLUMN '0' TO 'design_desc';

 * sqlite:///res_data.db
Done.
Done.


1


In [96]:
%sql SELECT * FROM design

 * sqlite:///res_data.db
Done.


design_code,design_desc
0,1 STORY - RANCH
1,1-STORY CONDO
2,1-STORY TWNHM
3,1-STORY TWNHM STYLE CONDO
4,2-3 STORY
5,A-FRAME
6,BI-LEVEL
7,DETACHED MULTI-STORY CONDO
8,DOUBLE WIDE
9,GARAGE DETACHED RESIDENTIAL


In [97]:
#Finally, convert rest of table data into its own table
data.to_sql('res_info', engine, index=False)

In [7]:
%sql SELECT * FROM res_info LIMIT 10

 * sqlite:///res_data.db
Done.


NCODE,ACCT#,PROPERTY_ADDRESS,LOCCITY,MULTIPLE_BLDGS,ACCOUNT_TYPE,DESCRIP,DESIGN,YEAR_BUILT,BEDROOMS,FULL_BATHS,THREE_QTR_BATHS,HALF_BATHS,ABOVE_GROUND_SQFT,FINISHED_BSMT_SQFT,UNFINISHED_BSMT_SQFT,GARAGE_SQFT,FINISHED_GARAGE_SQFT,STUDIO_SQFT,OTHER_BLDGS,SALE_DATE,SALE_PRICE,LAND_VALUE,BLDG_VALUE,EXTRA_FEATURE_VALUE
103,R0071420,1042 8TH ST,0,1,6,5,4,2000,5,5,0,1,2658,1117,625,462,0,330,0,2020-01-02,1842000,1035000,807000,0
107,R0000908,2385 4TH ST,0,0,6,5,4,1969,4,1,2,1,2430,617,0,536,0,0,0,2020-01-02,1750000,1102000,915600,0
155,R0110294,1486 SEIBERT CT,7,0,6,5,4,1991,4,1,2,0,2212,316,315,680,0,0,0,2020-01-02,610000,358000,256700,0
158,R0069754,1400 ROCKMONT CIR,0,0,6,5,18,1978,3,2,0,0,1082,552,0,460,0,0,0,2020-01-02,1200000,690000,266000,0
440,R0126278,1609 HOLEMAN DR,1,0,6,5,4,1997,4,2,1,1,1825,572,24,440,0,0,0,2020-01-02,450000,76000,385300,0
440,R0607799,945 GRENVILLE CIR,1,0,6,5,4,2019,6,2,2,1,4356,0,1088,764,0,0,0,2020-01-02,696900,99000,561100,0
440,R0608732,913 SANDSTONE CIR,1,0,6,5,4,2019,5,4,0,0,3084,0,1456,700,0,0,0,2020-01-02,654400,87000,491300,0
450,R0509875,1340 FOREST PARK CIR 2,2,0,7,0,1,2005,2,1,1,0,1720,0,0,504,0,0,0,2020-01-02,615000,0,464500,0
470,R0091942,957 CLOVER CIR,2,0,6,5,6,1983,4,1,1,0,924,484,0,440,0,0,0,2020-01-02,474000,139000,247000,0
135,R0113610,3260 47TH ST 105A,0,0,7,0,1,1984,1,1,0,0,757,0,0,153,0,0,0,2020-01-03,365000,0,361800,0


###Demonstrating simple join with new relational database:

In [11]:
%%sql
SELECT r.property_address, c.city_name, a.acct_type_desc
FROM res_info r
JOIN city c
ON r.loccity = c.city_code
JOIN account_type a
ON r.account_type = a.acct_type_code
LIMIT 10

 * sqlite:///res_data.db
Done.


PROPERTY_ADDRESS,city_name,acct_type_desc
1042 8TH ST,BOULDER,RESIDENTIAL
2385 4TH ST,BOULDER,RESIDENTIAL
1486 SEIBERT CT,SUPERIOR,RESIDENTIAL
1400 ROCKMONT CIR,BOULDER,RESIDENTIAL
1609 HOLEMAN DR,ERIE,RESIDENTIAL
945 GRENVILLE CIR,ERIE,RESIDENTIAL
913 SANDSTONE CIR,ERIE,RESIDENTIAL
1340 FOREST PARK CIR 2,LAFAYETTE,RESIDENTIAL CONDO
957 CLOVER CIR,LAFAYETTE,RESIDENTIAL
3260 47TH ST 105A,BOULDER,RESIDENTIAL CONDO
